In [5]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch

# import scripts
from src.Botorch_opt import bayesian_optimiser
from src.api_helper import api_utils
from src.api_helper import fake_api
from datetime import datetime as dt, date as dt_date

dtype = torch.float32

##### data

In [7]:
#  starting position
x0 = np.array([ 
    [10,21,39,12,14],
    [42,52,63,11,54],
    [71,82,99,42,57],
    [15,24,67,78,93]
]) 
N = np.sum(x0,axis=1)  #  row sum

In [8]:
transformer.x_to_y(X)

NameError: name 'transformer' is not defined

In [13]:
@api_utils.transform
def api(x): return wrap.f(x)

ndim = wrap.x0.shape[0] # input dimension
x0 = torch.from_numpy(wrap.x0).float().view(-1,ndim)
y0 = torch.tensor([[-1]], dtype=torch.float).view(-1,1)

print("input shape:",x0.size())
print("output shape:",y0.size())

input shape: torch.Size([1, 14])
output shape: torch.Size([1, 1])


##### OPT

In [19]:
# hyperparameters
T = 10 # total number of iterations

# gp; includes "MA2.5", "SE", "RQ", "LR", "PO"
gp_name,gp_params = "RQ",{
                          "mode": "pro",      # "raw", "add", "pro" for GPs
                          "opt":"ADAM",        # opt for MLE; includes: quasi_newton, ADAM
                          "epochs":128,       # if chosen ADAM as opt
                          "lr":1e-1,          # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 1

hyper_params = { 
    "acq_name" : "UCB",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

bayes_opt = bayesian_optimiser(gp_name,gp_params,hyper_params)

In [20]:
%%time
xs, ys = bayes_opt.outer_loop(T, x0, y0, m0,api,batch_size)

Iter 128 - Loss: 0.047
acq_func took 3.8s
api took 13.6s
time step 1, drop -6.63%; min $61,089,612
Iter 128 - Loss: -1.988
acq_func took 1.2s
api took 16.1s
time step 2, drop -7.56%; min $61,626,916
Iter 128 - Loss: -1.972
acq_func took 0.2s
api took 14.9s
time step 3, drop -42.35%; min $81,557,688
Iter 128 - Loss: -0.509
acq_func took 2.0s
api took 14.8s
time step 4, drop -0.37%; min $57,504,900
Iter 128 - Loss: -0.680
acq_func took 2.6s
api took 25.3s
time step 5, drop 3.47%; min $55,303,652
Iter 128 - Loss: -0.787
acq_func took 3.1s
api took 15.1s
time step 6, drop 2.71%; min $55,742,448
Iter 128 - Loss: -0.878
acq_func took 2.3s
api took 15.7s
time step 7, drop -9.76%; min $62,886,724
Iter 128 - Loss: -0.875
acq_func took 2.2s
api took 15.2s
time step 8, drop 0.59%; min $56,953,212
Iter 128 - Loss: -0.942
acq_func took 2.3s
api took 15.5s
time step 9, drop 1.29%; min $56,552,992
Iter 128 - Loss: -1.022
acq_func took 2.6s
api took 13.9s
time step 10, drop 2.40%; min $55,917,312
acq_

##### multiple runs

In [21]:
bayes_opt = bayesian_optimiser(gp_name,gp_params,hyper_params)
bayes_loop = exp_helper.multi_run_decorator(bayes_opt.outer_loop, runs=2)
xs,ys = bayes_loop(T, x0, y0, m0,api,batch_size)

Iter 128 - Loss: 0.047
acq_func took 2.5s
api took 31.0s
time step 1, drop -9.47%; min $62,717,960
Iter 128 - Loss: -1.631
acq_func took 1.4s
api took 16.0s
time step 2, drop 4.30%; min $54,828,644
Iter 128 - Loss: -1.528
acq_func took 3.1s
api took 15.4s
time step 3, drop 2.62%; min $55,792,464
Iter 128 - Loss: -1.759
acq_func took 2.1s
api took 18.9s
time step 4, drop 0.38%; min $57,075,612
Iter 128 - Loss: -1.873
acq_func took 1.8s
api took 11.8s
time step 5, drop 6.30%; min $53,685,988
Iter 128 - Loss: -1.922
acq_func took 3.6s
api took 10.6s
time step 6, drop 7.16%; min $53,191,664
Iter 128 - Loss: -1.992
acq_func took 2.1s
api took 10.9s
time step 7, drop 7.30%; min $53,109,124
Iter 128 - Loss: -2.054
acq_func took 1.8s
api took 10.3s
time step 8, drop 7.74%; min $52,856,408
Iter 128 - Loss: -2.095
acq_func took 2.5s
api took 10.3s
time step 9, drop 6.50%; min $53,568,292
Iter 128 - Loss: -2.145
acq_func took 5.2s
api took 10.2s
time step 10, drop 6.95%; min $53,311,068
acq_func 

In [22]:
len(xs)

2